##  An Agentic Workflow for Creating 3D Images from Natural Language

In [30]:
import numpy as np
import plotly.graph_objects as go
from typing_extensions import Literal
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel

from dotenv import load_dotenv

load_dotenv()

True

In [18]:
# Initialize Gemini 2.5 Pro preview model for both agents
model_name = "models/gemini-2.0-flash"
enhancer_llm = ChatGoogleGenerativeAI(model=model_name)
plotly_llm = ChatGoogleGenerativeAI(model=model_name)

In [19]:
# Define the state schema
class State(BaseModel):
    user_input: str = ""
    enhanced_description: str = ""
    plotly_code: str = ""

In [20]:
# System prompt for Agent 1: Enhance diagram description
enhancer_system_prompt = """You are a helpful assistant that takes a simple diagram description and enhances it with a very detailed description including 3D spatial layout, colors, and fine details. Provide a rich, vivid, and precise description."""

# System prompt for Agent 2: Generate Plotly code
plotly_system_prompt = """You are a Python developer assistant. Given a detailed diagram description, generate Plotly (version 6.0.1 or higher) Python code that renders the diagram. Return only the code without explanations."""

In [21]:
# Agent 1: Enhance diagram description
def agent1(state: State) -> Command[Literal["agent2"]]:
    messages = [
        SystemMessage(content=enhancer_system_prompt),
        HumanMessage(content=state.user_input),
    ]
    response = enhancer_llm.invoke(messages)
    return Command(
        goto="agent2",
        update={"enhanced_description": response.content}
    )

In [22]:
# Agent 2: Generate Plotly code from enhanced description
def agent2(state: State) -> Command[Literal[END]]:
    messages = [
        SystemMessage(content=plotly_system_prompt),
        HumanMessage(content=state.enhanced_description),
    ]
    response = plotly_llm.invoke(messages)
    return Command(
        goto=END,
        update={"plotly_code": response.content}
    )

In [23]:
# Build the multi-agent graph
builder = StateGraph(State)
builder.add_node(agent1)
builder.add_node(agent2)
builder.add_edge(START, "agent1")
builder.add_edge("agent1", "agent2")
builder.add_edge("agent2", END)

graph = builder.compile()

In [24]:
def run_graph(user_input: str) -> str:
    """
    Function to run the multi-agent graph with the provided user input.
    """
    initial_state = State(user_input=user_input)
    result = graph.invoke(initial_state)
    
    plotly_code = result["plotly_code"].strip("```python").strip("```").strip()

    return plotly_code


In [25]:
# natural language to 3D diagram function
def nl_to_3d(user_input: str):
    """
    Function to convert a diagram description to Plotly code.
    """
    plotly_code = run_graph(user_input)
    
    # Execute the generated Plotly code
    try:
        exec(plotly_code)
    except Exception as e:
        print(f"An error occurred while executing the Plotly code: {e}")


The 3D diagram is generated using the **"gemini-2.0-flash"** model, which powers both the enhancer and Plotly code generation agents in this workflow.


In [26]:
nl_to_3d("A diagram showing the layers of the earth. It includes the inner and outer cores, the mantle, and the crust.")

## Using Google AI Studio with Gemini-2.5-Pro Model

The **Gemini-2.5-Pro** model is currently one of the best coding models available. However, due to insufficient credits, it cannot be directly utilized in this workflow. 

To overcome this limitation, Google AI Studio can be leveraged by setting the system prompts and configurations correctly. By doing so, the model generates the desired code effectively without requiring direct access to the Gemini-2.5-Pro model.

here plotly_code 


In [27]:
def run_plotly_code(plotly_code: str):
    """
    Function to execute the Plotly code.
    """
    try:
        exec(plotly_code)
    except Exception as e:
        print(f"An error occurred while executing the Plotly code: {e}")
        

## Code Generated by "gemini-2.5-pro-preview-05-06"

The following code was directly generated using the **"gemini-2.5-pro-preview-05-06"** model. This model is known for its advanced capabilities in generating precise and efficient Python code for complex workflows. 


In [31]:
plotly_code = """ 

import plotly.graph_objects as go
import numpy as np

# Helper function to create a spherical cap surface
def create_spherical_cap(radius, color, name, n_points=50, theta_max_deg=270):
    
    theta_max_rad = np.deg2rad(theta_max_deg)
    phi = np.linspace(0, np.pi, n_points)  # Polar angle (from Z-axis)
    theta = np.linspace(0, theta_max_rad, n_points)  # Azimuthal angle (around Z-axis)
    phi, theta = np.meshgrid(phi, theta)

    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)

    # Using surfacecolor with a single color in colorscale for uniform color
    return go.Surface(
        x=x, y=y, z=z,
        name=name,
        surfacecolor=np.full_like(x, 0.5), # Dummy value, color defined by colorscale
        colorscale=[[0, color], [1, color]],
        showscale=False,
        hoverinfo='name'
    )

# Helper function to create a flat cut surface (a sector of a circle)
def create_cut_surface(radius_inner, radius_outer, theta_angle_rad, color, name, n_points_radial=10, n_points_angular=30):
    
    radii = np.linspace(radius_inner, radius_outer, n_points_radial)
    phi_cut = np.linspace(0, np.pi, n_points_angular) # This phi is for the circular sector's extent
    radii, phi_cut = np.meshgrid(radii, phi_cut)

    # Parametric equation for a disk sector in a plane defined by theta_angle_rad
    # For a plane at a specific theta, x and y are related.
    # We are essentially creating a sector in the XY plane (if theta=0) and then rotating it,
    # or more directly, calculating coordinates in 3D.
    # x = r * sin(phi_for_z_axis) * cos(theta_angle)
    # y = r * sin(phi_for_z_axis) * sin(theta_angle)
    # z = r * cos(phi_for_z_axis)
    # Here, 'radii' is our r, and 'phi_cut' is our phi_for_z_axis

    x = radii * np.sin(phi_cut) * np.cos(theta_angle_rad)
    y = radii * np.sin(phi_cut) * np.sin(theta_angle_rad)
    z = radii * np.cos(phi_cut)

    return go.Surface(
        x=x, y=y, z=z,
        name=name + " cut",
        surfacecolor=np.full_like(x, 0.5),
        colorscale=[[0, color], [1, color]],
        showscale=False,
        hoverinfo='name'
    )

# Define layer properties
# Using relative radii for simplicity in plotting
# Actual radii (approx km): Inner Core 1220, Outer Core 3480, Mantle 6340 (to base of crust), Crust 6371
# Scaled for visualization:
scale_factor = 1.0 # You can adjust this if needed
radii = {
    "Inner Core": 0.2 * scale_factor,
    "Outer Core": 0.4 * scale_factor,
    "Mantle": 0.9 * scale_factor, # Mantle is thick
    "Crust": 1.0 * scale_factor   # Outermost
}

colors = {
    "Inner Core": "rgb(220, 220, 200)", # Light grey/yellow
    "Outer Core": "rgb(255, 165, 0)",   # Orange/Yellow
    "Mantle": "rgb(200, 80, 30)",       # Reddish brown
    "Crust": "rgb(139, 69, 19)"         # Brown
}

fig = go.Figure()

theta_cut_max_deg = 270  # Degree of the spherical part shown (e.g. 270 for 3/4 sphere)
theta_cut_max_rad = np.deg2rad(theta_cut_max_deg)
theta_cut_start_rad = 0 # The other side of the cut

# Add layers from inside out
# 1. Inner Core (solid sphere, but we'll show it as a cap for consistency with cutaway)
fig.add_trace(create_spherical_cap(radii["Inner Core"], colors["Inner Core"], "Inner Core", theta_max_deg=theta_cut_max_deg))
# No inner radius for the innermost core's cut surfaces
fig.add_trace(create_cut_surface(0, radii["Inner Core"], theta_cut_start_rad, colors["Inner Core"], "Inner Core"))
fig.add_trace(create_cut_surface(0, radii["Inner Core"], theta_cut_max_rad, colors["Inner Core"], "Inner Core"))


# 2. Outer Core
fig.add_trace(create_spherical_cap(radii["Outer Core"], colors["Outer Core"], "Outer Core", theta_max_deg=theta_cut_max_deg))
fig.add_trace(create_cut_surface(radii["Inner Core"], radii["Outer Core"], theta_cut_start_rad, colors["Outer Core"], "Outer Core"))
fig.add_trace(create_cut_surface(radii["Inner Core"], radii["Outer Core"], theta_cut_max_rad, colors["Outer Core"], "Outer Core"))

# 3. Mantle
fig.add_trace(create_spherical_cap(radii["Mantle"], colors["Mantle"], "Mantle", theta_max_deg=theta_cut_max_deg))
fig.add_trace(create_cut_surface(radii["Outer Core"], radii["Mantle"], theta_cut_start_rad, colors["Mantle"], "Mantle"))
fig.add_trace(create_cut_surface(radii["Outer Core"], radii["Mantle"], theta_cut_max_rad, colors["Mantle"], "Mantle"))

# 4. Crust (very thin, exaggerated here for visibility)
fig.add_trace(create_spherical_cap(radii["Crust"], colors["Crust"], "Crust", theta_max_deg=theta_cut_max_deg))
fig.add_trace(create_cut_surface(radii["Mantle"], radii["Crust"], theta_cut_start_rad, colors["Crust"], "Crust"))
fig.add_trace(create_cut_surface(radii["Mantle"], radii["Crust"], theta_cut_max_rad, colors["Crust"], "Crust"))


# --- Labels (Annotations) ---
# Positioning annotations in 3D can be tricky and might need adjustment
# We place them on one of the cut faces, near the layer.
# For the 'y' coordinate of the annotation, we can use a small offset from the cut plane.
# The cut plane for labels will be at theta_cut_max_rad (the "front" cut)
label_x_factor = np.cos(theta_cut_max_rad + np.deg2rad(5)) # Slightly offset from the cut plane
label_y_factor = np.sin(theta_cut_max_rad + np.deg2rad(5)) # Slightly offset from the cut plane

annotations = [
    dict(
        showarrow=True, arrowhead=2, arrowwidth=1.5, arrowcolor="black",
        x=radii["Inner Core"] * 0.5 * label_x_factor, # Mid-radius of the layer
        y=radii["Inner Core"] * 0.5 * label_y_factor,
        z=0, # Centered vertically on the cut
        text="Inner Core",
        xanchor="left", yanchor="middle",
        font=dict(color="black", size=12)
    ),
    dict(
        showarrow=True, arrowhead=2, arrowwidth=1.5, arrowcolor="black",
        x=(radii["Inner Core"] + radii["Outer Core"]) * 0.5 * label_x_factor,
        y=(radii["Inner Core"] + radii["Outer Core"]) * 0.5 * label_y_factor,
        z=0.1 * scale_factor, # Slightly above center
        text="Outer Core",
        xanchor="left", yanchor="middle",
        font=dict(color="black", size=12)
    ),
    dict(
        showarrow=True, arrowhead=2, arrowwidth=1.5, arrowcolor="black",
        x=(radii["Outer Core"] + radii["Mantle"]) * 0.5 * label_x_factor,
        y=(radii["Outer Core"] + radii["Mantle"]) * 0.5 * label_y_factor,
        z=0.2 * scale_factor, # Further above center
        text="Mantle",
        xanchor="left", yanchor="middle",
        font=dict(color="black", size=12)
    ),
    dict(
        showarrow=True, arrowhead=2, arrowwidth=1.5, arrowcolor="black",
        x=(radii["Mantle"] + radii["Crust"]) * 0.5 * label_x_factor,
        y=(radii["Mantle"] + radii["Crust"]) * 0.5 * label_y_factor,
        z=0.3 * scale_factor, # Even further above center
        text="Crust",
        xanchor="left", yanchor="middle",
        font=dict(color="black", size=12)
    )
]


# --- Layout and Camera ---
fig.update_layout(
    title_text="Earth's Internal Layers (3D Cutaway)",
    scene=dict(
        xaxis=dict(visible=False, showbackground=False, showgrid=False, zeroline=False),
        yaxis=dict(visible=False, showbackground=False, showgrid=False, zeroline=False),
        zaxis=dict(visible=False, showbackground=False, showgrid=False, zeroline=False),
        aspectmode='data', # Ensures spheres look like spheres
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.0) # Adjust camera for a good view
        ),
        annotations=annotations
    ),
    margin=dict(l=10, r=10, b=10, t=50),
    # paper_bgcolor="lightgrey" # Optional: background color
)

# Improve lighting to make surfaces more distinct
fig.update_traces(lighting=dict(ambient=0.4, diffuse=0.8, specular=0.3, roughness=0.6))


fig.show()
"""

In [32]:
run_plotly_code(plotly_code)